In [1]:
!pip install torch datasets numpy matplotlib seaborn
!pip uninstall -y datasets fsspec huggingface_hub transformers tokenizers
!rm -rf ~/.cache/huggingface/datasets
!pip install datasets==2.14.7 fsspec==2023.10.0 huggingface_hub==0.17.3 transformers==4.35.2 tokenizers==0.15.0
!pip install torch-geometric

Defaulting to user installation because normal site-packages is not writeable
Found existing installation: datasets 2.14.7
Uninstalling datasets-2.14.7:
  Successfully uninstalled datasets-2.14.7
Found existing installation: fsspec 2023.10.0
Uninstalling fsspec-2023.10.0:
  Successfully uninstalled fsspec-2023.10.0
Found existing installation: huggingface-hub 0.17.3
Uninstalling huggingface-hub-0.17.3:
  Successfully uninstalled huggingface-hub-0.17.3
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
Found existing installation: tokenizers 0.15.0
Uninstalling tokenizers-0.15.0:
  Successfully uninstalled tokenizers-0.15.0
Defaulting to user installation because normal site-packages is not writeable
  Using cached datasets-2.14.7-py3-none-any.whl (520 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl (166 kB)
  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Using cached transformers-4.35.

In [2]:
#print(torch.__version__)
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.7.0+cu126.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.pyg.org/whl/torch-2.7.0+cu126.html


<h1>Optimized Test Run</h1>


In [ ]:
!python run_gnn_moe.py \
  --coupler_type HGNN \
  --static_hyperedge_strategy all_pairs \
  --hgnn_learnable_edge_weights \
  --embed_dim 512 \
  --num_layers 8 \
  --batch_size 16 \
  --learning_rate 0.0005 \
  --gnn_layers 3 \
  --num_experts 2 \
  --epochs 10 \
  --eval_every 250 \
  --run_name "hgnn_vs_gnn_optimal_showdown" \
  --dataset_config_name wikitext-2-v1


<h1>Ghost Ops</h1>

## Ghost Expert Architecture Training


In [17]:
# Ghost Expert Architecture Training
!python3 -m ghost.run_gnn_moe \
  --embed_dim 128 \
  --num_layers 4 \
  --num_experts 2 \
  --num_ghost_experts 4 \
  --ghost_activation_threshold 0.015 \
  --ghost_learning_rate 1e-4 \
  --epochs 3 \
  --max_batches_per_epoch 2 \
  --eval_every 50 \
  --run_name "ghost_notebook_demo" \
  --dataset_config_name wikitext-103-v1


/home/orion/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Adjusting num_heads from 8 to 4 based on embed_dim 256
🚀 Setting up data loading for wikitext / wikitext-103-v1...
/home/orion/.local/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)
Epoch 1/3:   0%|                                          | 0/2 [00:00<?, ?it/s]/home/orion/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the 

In [ ]:
# Test different ghost configurations
!python ghost/tests/run_ghost_sweep.py


In [ ]:
# Test different architecture configurations
!python ghost/tests/run_architecture_sweep.py


In [ ]:
import glob
import os

# Find latest sweep
sweep_dirs = glob.glob('ghost/tests/sweeps/*')
if sweep_dirs:
    latest_sweep = max(sweep_dirs, key=os.path.getctime)
    print(f"Analyzing: {latest_sweep}")
    !python ghost/tests/enhanced_analysis.py {latest_sweep}
else:
    print("No sweeps found")


### Load and Visualize

In [18]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# Load latest training log
log_files = glob.glob('ghost/tests/sweeps/*/*/training_log.json')
if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    with open(latest_log, 'r') as f:
        training_data = json.load(f)
    
    df = pd.DataFrame(training_data)
    
    # Plot ghost activations over time
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Ghost activations
    ghost_activations = pd.DataFrame(df['ghost_activations'].tolist(), index=df['step'])
    ghost_activations.plot(ax=ax1, title='Ghost Expert Activation Levels')
    ax1.set_ylabel('Activation Level')
    
    # Saturation vs Orthogonality
    ax2.plot(df['step'], df['saturation_level'], label='Saturation', color='red')
    ax2_twin = ax2.twinx()
    ax2_twin.plot(df['step'], df['orthogonality_score'], label='Orthogonality', color='blue')
    ax2.set_title('Expert Dynamics')
    ax2.set_xlabel('Training Step')
    ax2.legend(loc='upper left')
    ax2_twin.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Final ghost activations: {df.iloc[-1]['ghost_activations']}")
    print(f"Final saturation: {df.iloc[-1]['saturation_level']:.3f}")
else:
    print("No training logs found")


NameError: name 'glob' is not defined